In [11]:
from pathlib import Path
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    InverseDynamicsController,
    MeshcatPoseSliders,
    Context,
    RigidTransform,
    RotationMatrix,
    Sphere,
    Rgba,
    LeafSystem,
    RollPitchYaw,
    MultibodyPlant,
    Parser,
    Solve,
    SolutionResult,
    AddMultibodyPlantSceneGraph,
    AddFrameTriadIllustration,
    FixedOffsetFrame,
    TrajectorySource,
    Trajectory,
    PiecewisePose,
    PiecewisePolynomial,
    ConstantValueSource,
    AbstractValue
)
from pydrake.multibody import inverse_kinematics
from pydrake.multibody.inverse_kinematics import (
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsIntegrator,
)
from pydrake.trajectories import PiecewisePolynomial


from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)
from manipulation.utils import RenderDiagram, running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad

import numpy as np

### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [12]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


In [ ]:
hammer_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="hammer">
    <static>false</static>

    <link name="hammer_link">
        <!-- Rough inertial properties (good enough for simulation) -->
        <inertial>
            <mass>1.0</mass>
            <inertia>
                <ixx>0.01</ixx>
                <iyy>0.01</iyy>
                <izz>0.01</izz>
                <ixy>0.0</ixy>
                <ixz>0.0</ixz>
                <iyz>0.0</iyz>
            </inertia>
        </inertial>

        <!-- Handle: long thin cylinder, centered along +z -->
        <visual name="handle_visual">
            <pose>0 0 0.15 0 0 0</pose>
            <geometry>
            <cylinder>
                <radius>0.015</radius>
                <length>0.35</length>
            </cylinder>
            </geometry>
            <material>
            <ambient>0.75 0.55 0.3 1</ambient>
            <diffuse>0.75 0.55 0.3 1</diffuse>
            </material>
        </visual>

        <collision name="handle_collision">
            <pose>0 0 0.15 0 0 0</pose>
            <geometry>
            <cylinder>
                <radius>0.015</radius>
                <length>0.35</length>
            </cylinder>
            </geometry>
        </collision>

        <!-- Head: short, fat cylinder at the top of the handle -->
        <visual name="head_visual">
            <pose>0 0 0.3 0 1.5708 0</pose>
            <geometry>
            <cylinder>
                <radius>0.06</radius>
                <length>0.15</length>
            </cylinder>
            </geometry>
            <material>
            <ambient>0.75 0.55 0.3 1</ambient>
            <diffuse>0.75 0.55 0.3 1</diffuse>
            </material>
        </visual>

        <collision name="head_collision">
            <pose>0 0 0.3 0 1.5708 0</pose>
            <geometry>
            <cylinder>
                <radius>0.06</radius>
                <length>0.15</length>
            </cylinder>
            </geometry>
        </collision>
    </link>
    <!-- frame for hammer head (center of head) -->
    <frame name="hammer_head" attached_to="hammer_link">
        <pose>0 0 0.3 0 1.5708 0</pose>
    </frame>
    <frame name="hammer_face" attached_to="hammer_link">
        <pose>0.075 0 0.3 0 1.5708 0</pose>
    </frame>
  </model>
</sdf>
"""
# If using Deepnote, use this filepath (might need to manually create the assets directory)
# Write to file
# output_dir = Path("assets/")
# with open(output_dir / "hammer.sdf", "w") as f:
#   f.write(hammer_sdf)
# hammer_sdf_filepath = f"{Path.cwd()}/assets/hammer.sdf"
# print("Wrote hammer.sdf",hammer_sdf_filepath)

# If using github, use this filepath
hammer_sdf_filepath = Path.cwd().parent / "bonkbot" / "sim" / "assets" / "hammer.sdf"
print(hammer_sdf_filepath)

/workspaces/64212_projects/src/whackamole_project/whackamole/bonkbot/sim/assets/hammer.sdf


In [14]:
scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: hammer
    file: file://{hammer_sdf_filepath}
    default_free_body_pose:
        hammer_link:
            translation: [0, 0, 0]
            rotation: !Rpy {{ deg: [0, 0, 0] }}
- add_weld:
    parent: iiwa::iiwa_link_7
    child: hammer::hammer_link
    X_PC:
        translation: [0, 0, 0.06]
        rotation: !Rpy {{deg: [0, -90, 0] }}
- add_model:
    name: soup
    file: package://manipulation/hydro/005_tomato_soup_can.sdf
- add_weld:
    parent: world
    child: soup::base_link_soup
    X_PC:
        translation: [.5, -.8, 0.1]
        rotation: !Rpy {{ deg: [-90, 0, 0] }}
model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
"""

In [15]:
def get_prehit_pose(X_WO: RigidTransform, X_HL7: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    """
    Given the object pose X_WO, compute a prehit pose in hammer face frame and iiwa link 7 frame.
    
    Parameters:
        X_WO: pose of target object in world frame
        X_HL7: pose of iiwa link 7 in hammer face frame
    """
    p_OH = np.array([0.0, -0.2, 0.0])
    R_OH = RotationMatrix.MakeYRotation(-np.pi/2) @ RotationMatrix.MakeXRotation(-np.pi/2)
    X_OH = RigidTransform(R_OH, p_OH) # pose of hammer face from object frame
    X_WH_prehit = X_WO @ X_OH # prehit pose of hammer face 
    X_WL7_prehit = X_WH_prehit @ X_HL7
    return X_WH_prehit, X_WL7_prehit

def solve_ik(plant, plant_context, X_WH, q_guess=None, pos_tol=1e-3, theta_bound=1e-2):
    # Get the hammer face frame 
    hammer = plant.GetModelInstanceByName("hammer")
    hammer_face_frame = plant.GetFrameByName("hammer_face",hammer)
    W = plant.world_frame()

    if q_guess is None:
        iiwa = plant.GetModelInstanceByName("iiwa")
        iiwa_q0 = plant.GetPositions(plant_context, iiwa)

    ik = inverse_kinematics.InverseKinematics(plant, plant_context)

    # Extract desired position/orientation from X_WG
    R_WH_desired = X_WH.rotation()
    p_WH_desired = X_WH.translation()

    # Position constraint: place origin of hammer face at p_WH_desired (within a small box)
    ik.AddPositionConstraint(
        frameA=W,
        p_BQ=np.zeros(3),
        frameB=hammer_face_frame,
        p_AQ_lower=p_WH_desired - pos_tol,
        p_AQ_upper=p_WH_desired + pos_tol,
    )

    # Orientation constraint: align world and hammer_face_frame up to some tolerance
    ik.AddOrientationConstraint(
        frameAbar=W,
        R_AbarA=RotationMatrix(),      # identity
        frameBbar=hammer_face_frame,
        R_BbarB=R_WH_desired,
        theta_bound=theta_bound,              # rad
    )

    # Add collision constraint to ensure collision free pre-hit pose
    # ik.AddMinimumDistanceConstraint(0.02)

    prog = ik.prog()
    prog.SetInitialGuess(ik.q(), q_guess)
    result = Solve(prog)

    if result.get_solution_result() != SolutionResult.kSolutionFound:
        return result.GetSolution(ik.q()), False
    return result.GetSolution(ik.q()), True


def make_joint_space_position_trajectory(path, timestep=1.0):
    traj = None
    times = [timestep * i for i in range(len(path))]
    Q = np.column_stack(path)
    traj = PiecewisePolynomial.FirstOrderHold(times, Q)
    return traj

In [16]:
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder = DiagramBuilder()
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")

temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)

X_WGinitial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("iiwa_link_7"))
print(X_WGinitial)
# Get hammer head frame
hammer = plant.GetModelInstanceByName("hammer")
hammer_face_frame = plant.GetFrameByName("hammer_face", hammer)
X_WHammer = hammer_face_frame.CalcPoseInWorld(temp_plant_context)

# Get soup pose
soup = plant.GetModelInstanceByName("soup")
soup_body = plant.GetBodyByName("base_link_soup", soup)
X_WSoup = plant.EvalBodyPoseInWorld(temp_plant_context, soup_body)

# get pose of iiwa link 7 in hammer face frame
iiwa = plant.GetModelInstanceByName("iiwa")
l7_frame = plant.GetFrameByName("iiwa_link_7", iiwa)
X_HL7 = plant.CalcRelativeTransform(temp_plant_context, hammer_face_frame, l7_frame)
# get pre-hit pose frame
X_WH_prehit, X_WL7_prehit = get_prehit_pose(X_WSoup, X_HL7)
print("X_WH_prehit",X_WH_prehit)
print("X_WL7_prehit",X_WL7_prehit)
# visualize extra prehit frames
AddMeshcatTriad(
    meshcat,
    path="hammer_prehit_pose_triad",  
    length=0.1,
    radius=0.005,
    X_PT=X_WH_prehit,                 
)
AddMeshcatTriad(
    meshcat,
    path="iiwa_prehit_pose_triad",  
    length=0.1,
    radius=0.005,
    X_PT=X_WL7_prehit,                 
)

# Visualize axes (useful for debugging)
scenegraph = station.GetSubsystemByName("scene_graph")
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName("base_link_soup"),
    length=0.1,
)
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    frame=hammer_face_frame,
    length=0.1,
)
AddFrameTriadIllustration(
    scene_graph=scenegraph, body=plant.GetBodyByName("iiwa_link_7"), length=0.1
)

# Get initial positions of the iiwa joints
iiwa_q0 = plant.GetPositions(temp_plant_context, iiwa)
print("iiwa q0", iiwa_q0)

# solve ik for goal joint config
q_goal, optimal = solve_ik(plant, temp_plant_context, X_WH_prehit, q_guess=iiwa_q0,
                    pos_tol=1e-6, theta_bound=1e-3)
print(q_goal, optimal)

# linear joint space position trajectory
path = [iiwa_q0, q_goal]
traj = make_joint_space_position_trajectory(path)
iiwa_src = builder.AddSystem(TrajectorySource(traj))
builder.Connect(iiwa_src.get_output_port(), station.GetInputPort("iiwa.position"))

diagram = builder.Build()

RigidTransform(
  R=RotationMatrix([
    [-0.000773199921913291, 0.999999682931835, 0.00019052063137842191],
    [0.9709578572896667, 0.0007963267107333862, -0.23924925335563643],
    [-0.23924932921398254, 2.0149982078937825e-16, -0.9709581651495918],
  ]),
  p=[0.0003536363619349402, -0.444084375221239, 0.7667078137695524],
)
X_WH_prehit RigidTransform(
  R=RotationMatrix([
    [6.123233995736766e-17, 1.0, -6.123233995736766e-17],
    [1.0, -6.123233995736766e-17, 6.123233995736766e-17],
    [6.123233995736766e-17, -6.123233995736766e-17, -1.0],
  ]),
  p=[0.5, -0.8, 0.30000000000000004],
)
X_WL7_prehit RigidTransform(
  R=RotationMatrix([
    [1.9311790677830432e-17, 1.0000000000000009, 2.5554204784949305e-17],
    [0.9999999999932544, -1.0326036587962048e-16, -3.6732051029385e-06],
    [-3.673205103854434e-06, -1.4796472243047925e-16, -0.9999999999932563],
  ]),
  p=[0.5, -0.4999995041193349, 0.43499889803755853],
)
iiwa q0 [-1.57  0.1   0.   -1.2   0.    1.6   0.  ]
[-0.91586871  

In [17]:
simulator = Simulator(diagram)

ctx = simulator.get_mutable_context()
diagram.ForcedPublish(ctx)

meshcat.StartRecording()

if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)

simulator.AdvanceTo(traj.end_time() if running_as_notebook else 5)
meshcat.StopRecording()
meshcat.PublishRecording()
input("Press Enter to exit...")  # keep Meshcat alive so you can look around

''

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e09b4ee5-e97d-4566-8819-f89ba4350a2e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>